<a href="https://colab.research.google.com/github/samuel-c-santos/Sentinel-Time-Viewer/blob/main/GEE_swipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q earthengine-api
!pip install -q geemap
!pip install -q ipywidgets
!pip install -q pycrs
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
import ee
import geemap
import datetime
import zipfile
import os
from google.colab import files
from IPython.display import display

# Autenticar e inicializar
ee.Authenticate()
ee.Initialize(project='ee-samuelsantosambientalcourse')

# 1) Upload e obtenção da geometria
def upload_shapefile():
    if os.path.exists("shapefile"):
        for f in os.listdir("shapefile"):
            os.remove(os.path.join("shapefile", f))
    else:
        os.makedirs("shapefile")
    print("📂 Faça upload de um arquivo .zip com o shapefile...")
    uploaded = files.upload()
    for fn in uploaded:
        if fn.endswith(".zip"):
            with zipfile.ZipFile(fn, "r") as z:
                z.extractall("shapefile")
            print("✅ Shapefile extraído.")
            break
    shp = [f for f in os.listdir("shapefile") if f.endswith(".shp")]
    if not shp:
        print("❌ .shp não encontrado."); return None
    return geemap.shp_to_ee(os.path.join("shapefile", shp[0])).geometry()

# … (imports e upload_shapefile continuam iguais) …

# 1) Função de download sem clip (como antes)
def get_sentinel_image(region, year, start_month=1, end_month=12):
    start_date = datetime.date(year, start_month, 1)
    end_date = datetime.date(year, end_month, 28 if end_month == 2 else 30)
    coll = (ee.ImageCollection("COPERNICUS/S2_SR")
            .filterBounds(region)
            .filterDate(str(start_date), str(end_date))
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))
    if coll.size().getInfo() == 0:
        empty = ee.Image.constant([0,0,0]).rename(['B4','B3','B2'])
        return empty.set('year', year)
    return coll.median().select(['B4','B3','B2']).set('year', year)

# 2) Função corrigida de rasterizar o contorno
def rasterizar_contorno(region, color='FF0000', width=3):
    fc = ee.FeatureCollection([ee.Feature(region)])
    # paint espera um FeatureCollection
    outline = ee.Image().byte().paint(fc, 1, width)
    return outline.visualize(palette=[color], opacity=1)

# 3) Execução principal, tudo dentro do if
feature = upload_shapefile()
if feature:
    region = feature
    Map = geemap.Map()
    Map.centerObject(region, zoom=15)

    anos = list(range(2016, datetime.datetime.now().year + 1))
    vis_params = {'min':0, 'max':3000, 'bands':['B4','B3','B2']}

    # 4) Montar lista de imagens já com contorno “embutido”
    images = []
    for yr in anos:
        base = get_sentinel_image(region, yr)
        rgb  = base.visualize(**vis_params)
        cont = rasterizar_contorno(region, color='00FF00', width=2)
        combo = ee.ImageCollection([rgb, cont]).mosaic().set('year', yr)
        images.append(combo)

    # 5) Passar essa coleção com contorno ao ts_inspector
    Map.ts_inspector(
        left_ts=ee.ImageCollection(images),
        right_ts=ee.ImageCollection(images),
        left_names=[str(y) for y in anos],
        right_names=[str(y) for y in anos],
        left_vis={}, right_vis={},
        width='80px'
    )

    Map.add_text(
        text='Comparação Sentinel‑2 por ano',
        position='bottomright',
        font_size='16pt', font_color='white', font_weight='bold'
    )

    display(Map)